In [481]:
import numpy as np

In [482]:
#test data
D=np.array([[-3,1],[1,-2]]);
l=np.array([[2],[3]]);
b=np.array([[1],[12]]);
Mat=np.array([[1,1],[3,4]]);
inq=np.array([1,1]);

In [483]:
def wolf(D,l,b,Mat,*args):
    n=l.size;
    m=b.size;
    
    
    
    #check no of arguments mismatch
    varargin = args;
    no_args=4+len(varargin);
    
    
    if no_args>6:
        print("\nError:Number of input arguments are inappropriate!\n");
        return
    if no_args <5:
        minimize=0;
        inq=-np.ones([m,1]);
    if no_args ==5:
        minimize=0;
        inq=np.asarray(varargin[0]);
    if no_args==6:
        inq=np.asarray(varargin[0]);
        minimize=varargin[1];
    if minimize == 1:
        l=-l;
        D=-D;
    
    #check dimension mismatch
    
    #if np.logical_not(np.size(Mat,1) == m) or np.logical_not(np.size(inq) == m) or np.logical_not(np.size(D,1) == np.size(D,2)) or np.logical_not(np.size(D,1) == n) or np.logical_not(np.size(Mat,2) == n):
     #   print('\\nError: Dimension mismatch!\\n')
      #  return x,fval
    
    
    #check convexity of Hessian
    s,s2=np.linalg.eig(-D);
    if np.ndarray.min(s) < 0:
        print('\\nError: Wolf method may not converge to global optimum!\\n')
        return x,fval
    else:
        if (np.ndarray.min(s) == 0) and np.logical_not(isempty(find(l,1))):
            print('\\nError: Wolf method may not converge to global optimum!\\n')
            return x,fval
    
    count=n
    for i in np.arange(0,m-1).reshape(-1):
        if (inq[i] > 0):
            Mat[i]=- Mat[i]
            b[i]=- b[i]
        else:
            if (inq[i] == 0):
                count=count + 1
                Mat[i,count]=- 1
                l[count]=0
                D[count,count]=0
    
    
    #a= np.array([[-2*D, Mat.transpose(), np.eye(count), np.zeros((count,m))], [Mat, np.zeros((m,m+count)), np.eye(m)]]);    d=np.array([[l],[b]]);
    d=np.array([[l],[b]]);
    #y=np.array([[-2*D, Mat.transpose(), np.eye(count), np.zeros((count,m))]])
    y=np.array([[-2*D, Mat, -np.eye(count), np.zeros((count,m))]])
    y=y.reshape(D.shape[0]+Mat.shape[0]+count+count,m)
    y=y.transpose();
    x=np.array([[Mat.transpose(), np.zeros((m,m)), np.zeros((m,count)), np.eye(m)]])
    x=x.reshape(D.shape[0]+Mat.shape[0]+count+count,m)
    x=x.transpose();
    a=np.concatenate((y, x), axis=0)
    
    
    
    
    d=d.reshape(d.size, 1)
    for i in np.arange(0,count+m).reshape(-1):
        if (d[i] < 0):
            d[i]=- d[i]
            a[i]=- a[i]
    
    cb=np.zeros([1,count + m])
    bv=np.zeros([1,count + m])
    nbv=(np.arange(1,np.dot(2,(count + m))))
    c=np.zeros([1,np.dot(2,(count + m))])
    rem=np.zeros([1,count + m])

    for i in np.arange(0,count+m).reshape(-1):
        if (a[i,count + m + i] == - 1):
            bv[0,i]=2*count + m + i
            cb[0,i]=- 1
        else:
            if (a[i,count + m + i] == 1):
                rem[0,i]=count + m + i
                bv[0,i]=count + m + i
                cb[0,i]=0
    
    bv=np.asarray(bv[0])
    k=(rem[np.nonzero(rem)])[0]
    
    
    a = (np.delete(a, k, axis=0))
    
    c = (np.delete(c, k, axis=0))
    
    nbv = (np.delete(nbv, k, axis=0))
    r=np.dot(cb,a) - c
    exitflg=0
    iter=0
    z=np.dot(cb,d)
    #w,y=np.size(a,nargout=2)
    w=a.shape[0];
    y=a.shape[1];
    opt=0
    while (exitflg == 0):

        iter=iter + 1
        print('\\n\\n %d th tableau:\\n',iter)
        print('\\n\\t\\t\\tBV\\t')
        print(nbv)
        
        r_new=r
        found=0
        
        while found == 0:

            
            v = np.unravel_index(np.argmin(r_new, axis=None), r_new.shape);
            u=r_new[v];
            v=v[0];
            leave=0
            if np.logical_not((u < 0)):
                if np.abs(z) > 10 ** - 6:
                    print('\\nError: Wolf method fails to find optimum!\\n')
                    exitflg=1
                    found=1
                else:
                    print('\\nThe optimum has achieved!\\n')
                    exitflg=1
                    opt=1
                    found=1
            else:
                ratio=np.left_shift(1,30); 
                check=0
                
                
                for i in np.arange(0,w).reshape(-1):
                    
                    
                    print(nbv[v]-bv[i]);
                    if bv[i] <= 2*(count + m) and np.abs(bv[i] - nbv[v]) == count + m:
                        check=1
                if check == 0:
                    for i in np.arange(0,w).reshape(-1):
                        if a[i,v] > 0 and (d[i] / a[i,v]) < ratio:
                            ratio=d[i] / a[i,v]
                            leave=np.copy(i)
                    print('\\nEntering Variable:')
                    print(nbv[v])
                    print('\\nLeaving Variable:')
                    print(bv[leave])
                    for i in np.arange(0,w).reshape(-1):
                        for j in np.arange(0,y).reshape(-1):
                            if i != leave and j != v:
                                a[i,j]=a[i,j] - np.dot(a[i,v],a[leave,j]) / a[leave,v]
                    
                    z=z - np.multiply(d[leave],r[v]) / a[leave,v]
                    for j in range(0,y):
                        if j != v:
                            print("rj")
                            print(r)
                            print(v)
                            print(r[v])
                            r[0,j]=r[0,j] - np.dot(r[0,v],a[leave,j]) / a[leave,v]
                            a[leave,j]=a[leave,j] / a[leave,v]
                    for i in np.arange(1,w).reshape(-1):
                        if i != leave:
                            d[i]=d[i] - np.dot(a[i,v],d[leave]) / a[leave,v]
                            a[i,v]=- a[i,v] / a[leave,v]
                    d[leave]=d[leave] / a[leave,v]
                    a[leave,v]=1 / a[leave,v]
                    r[v]=- r[v] /a[leave,v]
                    temp=nbv[v]
                    nbv[v]=bv[leave]
                    bv[leave]=temp
                    found=1

                else:
                    if check == 1:
                        r_new[v]=1


    
    if opt == 1:
        x=np.zeros(n,1)
        for i in np.arange(0,w).reshape(-1):
            if bv[i] <= n:
                x[bv[i]]=d[i]
        #fval=np.dot(x.transpose(),l) + np.dot(np.dot(x.T,D),x)
        fval=np.dot(x.transpose(),l) + np.dot(np.dot(x.transpose(),D),x)
        if minimize == 1:
            fval=- fval
    
    
    
    return x,fval

In [ ]:
wolf(D,l,b,Mat,inq)